In [13]:
# Imports for Project
import os
import numpy as np

Preparing Data

In [14]:
base_dir = os.getcwd()
data_path = f"{base_dir}/data/"
os.listdir(data_path)
print(data_path)

import joblib
from skimage.io import imread # loads an image from a file
from skimage.transform import resize # resizes an image to a particular size
 
def resize_all(src, pklname, include, width=227, height=227): # These should already be 227 pixels
    """
    load images from path, resize them and write them as arrays to a dictionary, 
    together with labels and metadata. The dictionary is written to a pickle file 
    named '{pklname}_{width}x{height}px.pkl'.
     
    Parameter
    ---------
    src: str
        path to data
    pklname: str
        path to output file
    width: int
        target width of the image in pixels
    include: set[str]
        set containing str
    """
     
    height = height if height is not None else width
     
    data = dict() #dictionary mapping
    data['label'] = []
    data['filename'] = []
    data['data'] = []   
     
    pklname = f"{pklname}_{width}x{height}px.pkl"
 
    # read all images in PATH, resize and write to DESTINATION_PATH
    for subdir in os.listdir(src):
        if subdir in include:
            print(subdir)
            current_path = os.path.join(src, subdir)
 
            for file in os.listdir(current_path):
                if file[-3:] in {'jpg', 'png'}:
                    im = imread(os.path.join(current_path, file))
                    im = resize(im, (width, height)) #[:,:,::-1]
                    data['label'].append(subdir[:-4])
                    data['filename'].append(file)
                    data['data'].append(im)
 
        joblib.dump(data, pklname)

c:\Users\colli\CompSci\PatternProject\CrackDetection/data/


In [15]:
# Resize all
base_name = 'GausNHogNOtsutest'
width = 227
include = {'Negative', 'Positive'}
resize_all(src=data_path, pklname=base_name, width=width, include=include)

from collections import Counter
 
data = joblib.load(f'{base_name}_{width}x{width}px.pkl')
 
print('number of samples: ', len(data['data']))
print('keys: ', list(data.keys()))
print('image shape: ', data['data'][0].shape)
print('labels:', np.unique(data['label']))
 
Counter(data['label'])

Negative
Positive
number of samples:  1569
keys:  ['label', 'filename', 'data']
image shape:  (227, 227, 3)
labels: ['Nega' 'Posi']


Counter({'Nega': 792, 'Posi': 777})

In [16]:
X = np.array(data['data'])
y = np.array(data['label'])

from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    shuffle=True,
    random_state=42,
)

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog
import skimage
import cv2
 
class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])

class GaussianBlurTransformer():
    def __init__(self) -> None:
        pass

    def transform(self, X, y=None):
        return np.array([skimage.filters.gaussian(img, sigma=1.0) for img in X])
        # skimage.filters.gaussian(gray_image, sigma=1.0)
        
        

class ThresholdTransformer(): #I think this is going to return a mask which is going to be passed in to the classifier. The classifier can do the rest?
    def __init__(self) -> None:
        pass

    def regenerate_img(self, img, threshold):
        row, col = img.shape 
        y = np.zeros((row, col))
        for i in range(0,row):
            for j in range(0,col):
                if img[i,j] >= threshold:
                    y[i,j] = 1
                else:
                    y[i,j] = 0
        return y

    def transform(self, X, y=None):
        tempArr = []
        for img in X: 
            t = skimage.filters.threshold_otsu(img)
            img = self.regenerate_img(img, t)
            tempArr.append(img)
        # print(tempArr.shape)
        # print(tempArr.ndim)
        return np.array(tempArr)
        #skimage.filters.threshold_otsu(blurred_image)

class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])


In [18]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

grayify = RGB2GrayTransformer()
GBlur = GaussianBlurTransformer()
TTrans = ThresholdTransformer()
hogify = HogTransformer(
    pixels_per_cell=(14, 14), 
    cells_per_block=(4,4), #2,2
    orientations=9, 
    block_norm='L2-Hys'
)
scalify = StandardScaler()

X_train_gray = grayify.fit_transform(X_train)
X_train_blur = GBlur.transform(X_train_gray)
X_train_otsu = TTrans.transform(X_train_blur)
X_train_hog = hogify.fit_transform(X_train_otsu)
X_train_prepared = scalify.fit_transform(X_train_hog)
#X_train_threshold = TTrans.transform(X_train_blur)
#X_train_prepared = scalify.fit_transform(X_train_blur)



In [19]:

# print(X_train.shape)
# print(X_train.ndim)
# print(X_train_gray.shape)
# print(X_train_gray.ndim)
# print(X_train_hog.shape)
# print(X_train_hog.ndim)
# print(X_train_threshold.shape)
# print(X_train_threshold.ndim)
print(X_train_hog.shape)

(1255, 24336)


In [20]:
from sklearn.linear_model import SGDClassifier #Stochastic Gradient Descent - incrementally trained logistic regression
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression #Logistic Regression is same as Stochastic Gradient Descent
from sklearn.svm import SVC

sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train_prepared, y_train)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train_prepared, y_train)
kmeans.labels_
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train_prepared, y_train)
svc_clf = SVC(gamma = 'auto')
svc_clf.fit(X_train_prepared, y_train)



SVC(gamma='auto')

In [38]:

X_test_gray = grayify.fit_transform(X_test)
X_test_blur = GBlur.transform(X_test_gray)
X_test_otsu = TTrans.transform(X_test_blur)
X_test_hog = hogify.fit_transform(X_test_otsu)
X_test_prepared = scalify.fit_transform(X_test_hog)
# X_test_threshold = TTrans.transform(X_test_blur)


ValueError: not enough values to unpack (expected 2, got 1)

In [22]:
y_pred = sgd_clf.predict(X_test_prepared)
# print(np.array(y_pred == y_test)[:10])
# print('')
print('Percentage correct Stochastic Gradient Descent: ', 100*np.sum(y_pred == y_test)/len(y_test))

ky_pred = kmeans.predict(X_test_prepared)
tempArr = []
for p in range(len(ky_pred)):
    tempArr.append(str(ky_pred[p]))

for l in range(len(tempArr)):
    if tempArr[l] == "0":
        tempArr[l] = "Posi"
    else: 
        tempArr[l] = "Nega"
print('')
print('Percentage correct KMeans: ', 100*np.sum(tempArr == y_test)/len(y_test))

lr_pred = lr_clf.predict(X_test_prepared)
print('')
print('Percentage correct Logistic Regression: ', 100*np.sum(lr_pred == y_test)/len(y_test))

svc_pred = svc_clf.predict(X_test_prepared)
print('')
print('Percentage correct Support Vector Classifier: ', 100*np.sum(svc_pred == y_test)/len(y_test))
 

Percentage correct Stochastic Gradient Descent:  94.26751592356688

Percentage correct KMeans:  4.777070063694268

Percentage correct Logistic Regression:  95.22292993630573

Percentage correct Support Vector Classifier:  95.54140127388536


In [23]:
from sklearn.model_selection import GridSearchCV

#Want to use cross validation to see if i can get better results.

In [24]:
from PIL import Image
def predict_on_crops(input_image, height=227, width=227, save_crops = False):
    im = cv2.imread(input_image)
    imgheight, imgwidth, channels = im.shape
    k=0
    output_image = np.zeros_like(im)
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            a = im[i:i+height, j:j+width]
            print(a)
            a_test_gray = grayify.fit_transform(a)
            a_test_blur = GBlur.transform(a_test_gray)
            a_test_otsu = TTrans.transform(a_test_blur)
            a_test_hog = hogify.fit_transform(a_test_otsu)
            a_test_prepared = scalify.fit_transform(a_test_hog)
            ## discard image cropss that are not full size
            # predicted_class = predict(base_model,Image.fromarray(a))
            # print(Image.fromarray(a))
            svc_pred_real = svc_clf.predict(a_test_prepared) 
            ## save image
            file, ext = os.path.splitext(input_image)
            image_name = file.split('/')[-1]
            folder_name = 'out_' + image_name
            ## Put predicted class on the image
            if svc_pred_real == 'Positive':
                color = (0,0, 255)
            else:
                color = (0, 255, 0)
            cv2.putText(a, svc_pred_real, (50,50), cv2.FONT_HERSHEY_SIMPLEX , 0.7, color, 1, cv2.LINE_AA) 
            b = np.zeros_like(a, dtype=np.uint8)
            b[:] = color
            add_img = cv2.addWeighted(a, 0.9, b, 0.1, 0)
            ## Save crops
            if save_crops:
                if not os.path.exists(os.path.join('real', folder_name)):
                    os.makedirs(os.path.join('real', folder_name))
                filename = os.path.join('real', folder_name,'img_{}.png'.format(k))
                cv2.imwrite(filename, add_img)
            output_image[i:i+height, j:j+width,:] = add_img
            k+=1
    ## Save output image
    cv2.imwrite(os.path.join('real','predictions', folder_name+ '.jpg'), output_image)
    return output_image

In [25]:
# from patchify import patchify, unpatchify

# im = imread('real/cracking-in-concrete.jpg')
# print(im.shape)
# patches = patchify(im, (227,227), step=1)

In [54]:
from PIL import Image
from numpy import asarray, dtype
def predict_on_crops(input_image, height=227, width=227, save_crops = False):
    im = Image.open('real/cracking-in-concrete.jpg')
    im = asarray(im) #write PIL as numpy array
    tiles = dict()
    tiles['data'] = []
    for x in range(0,im.shape[0],height):
        for y in range(0,im.shape[1],width):
            tile = im[x:x+height,y:y+width]
            tiles['data'].append(tile)
    # patches = np.array(tiles)        
    # print(patches.shape)
    print(len(tiles['data']))
    print(tiles['data'])
    a = np.array(tiles['data'])
    print(a)
    a_test_gray = grayify.fit_transform(a)
    a_test_blur = GBlur.transform(a_test_gray)
    a_test_otsu = TTrans.transform(a_test_blur)
    a_test_hog = hogify.fit_transform(a_test_blur)
    a_test_prepared = scalify.fit_transform(a_test_hog)
    svc_pred_real = svc_clf.predict(a_test_prepared) 

    for images in svc_pred_real:
        if svc_pred_real == 'Positive':
            color = (0,0, 255)
        else:
            color = (0, 255, 0)
        cv2.putText(images, svc_pred_real, (50,50), cv2.FONT_HERSHEY_SIMPLEX , 0.7, color, 1, cv2.LINE_AA) 
            

predict_on_crops('real/cracking-in-concrete.jpg', 227, 227)

15
[array([[[ 43,  45,  34],
        [ 44,  46,  35],
        [ 53,  55,  44],
        ...,
        [130, 130, 122],
        [120, 120, 112],
        [100, 100,  92]],

       [[ 41,  43,  32],
        [ 42,  44,  33],
        [ 46,  48,  37],
        ...,
        [130, 130, 122],
        [138, 138, 130],
        [131, 131, 123]],

       [[ 46,  48,  37],
        [ 50,  52,  41],
        [ 49,  51,  40],
        ...,
        [132, 132, 122],
        [145, 145, 135],
        [146, 146, 136]],

       ...,

       [[168, 170, 167],
        [161, 163, 160],
        [157, 159, 156],
        ...,
        [164, 163, 158],
        [161, 160, 155],
        [169, 168, 163]],

       [[166, 168, 165],
        [172, 174, 171],
        [173, 175, 172],
        ...,
        [148, 147, 142],
        [178, 177, 172],
        [157, 156, 151]],

       [[155, 157, 154],
        [171, 173, 170],
        [175, 177, 174],
        ...,
        [176, 175, 170],
        [188, 187, 182],
        [179, 178, 1

C:\Users\colli\AppData\Local\Temp\ipykernel_14868\136547363.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(tiles['data'])
C:\Users\colli\AppData\Local\Temp\ipykernel_14868\1560604694.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([skimage.color.rgb2gray(img) for img in X])
C:\Users\colli\AppData\Local\Temp\ipykernel_14868\1560604694.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is dep

ValueError: setting an array element with a sequence.

In [33]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
# output_image = predict_on_crops('real/concrete_crack1.jpg', 128, 128)
output_image = predict_on_crops('real/cracking-in-concrete.jpg', 128, 128)
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))

[[[ 34  45  43]
  [ 35  46  44]
  [ 44  55  53]
  ...
  [ 98 101 105]
  [102 103 107]
  [106 105 109]]

 [[ 32  43  41]
  [ 33  44  42]
  [ 37  48  46]
  ...
  [ 94  99 102]
  [102 103 107]
  [113 112 116]]

 [[ 37  48  46]
  [ 41  52  50]
  [ 40  51  49]
  ...
  [ 93  98 101]
  [101 104 108]
  [113 114 118]]

 ...

 [[145 142 137]
  [153 150 145]
  [159 159 153]
  ...
  [175 180 178]
  [174 177 175]
  [194 195 193]]

 [[144 141 136]
  [150 147 142]
  [153 153 147]
  ...
  [144 149 147]
  [158 161 159]
  [181 182 180]]

 [[149 146 141]
  [154 151 146]
  [157 157 151]
  ...
  [165 170 168]
  [175 178 176]
  [181 182 180]]]


ValueError: not enough values to unpack (expected 2, got 1)

<Figure size 720x720 with 0 Axes>